In [11]:
import pandas as pd

# Load dataset

In [12]:
general_data = pd.read_csv('../data/raw/general_data.csv')
employee_survey_data = pd.read_csv('../data/raw/employee_survey_data.csv')
manager_survey_data = pd.read_csv('../data/raw/manager_survey_data.csv')
in_time = pd.read_csv('../data/raw/in_time.csv')
out_time = pd.read_csv('../data/raw/out_time.csv')

print("===== General Data =====")
print(general_data.info())
print(general_data.head())

print("===== Employee Survey Data =====")
print(employee_survey_data.info())
print(employee_survey_data.head())

print("===== Manager Survey Data =====")
print(manager_survey_data.info())
print(manager_survey_data.head())

print("===== In Time =====")
print(in_time.info())
print(in_time.head())

print("===== Out Time =====")
print(out_time.info())
print(out_time.head())


===== General Data =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   EmployeeCount            4410 non-null   int64  
 8   EmployeeID               4410 non-null   int64  
 9   Gender                   4410 non-null   object 
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   object 
 12  MaritalStatus            4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14 

# Fix column

In [13]:
in_time = in_time.rename(columns={'Unnamed: 0': 'EmployeeID'})
out_time = out_time.rename(columns={'Unnamed: 0': 'EmployeeID'})

# Merge table

In [14]:
df = (
    general_data
    .merge(employee_survey_data, on='EmployeeID', how='left')
    .merge(manager_survey_data, on='EmployeeID', how='left')
)

In [15]:
in_dt = in_time.copy()
out_dt = out_time.copy()

for col in in_dt.columns[1:]:
    in_dt[col] = pd.to_datetime(in_dt[col])
for col in out_dt.columns[1:]:
    out_dt[col] = pd.to_datetime(out_dt[col])

work_hours = (out_dt.iloc[:, 1:] - in_dt.iloc[:, 1:]).apply(
    lambda x: x.dt.total_seconds() / 3600
)

avg_work_hours = work_hours.mean(axis=1)

work_hours = pd.concat(
    [in_dt[['EmployeeID']], avg_work_hours.rename('AvgWorkHours')],
    axis=1
).copy()

df = df.merge(work_hours, on='EmployeeID', how='left')

In [16]:
df

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,AvgWorkHours
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,6,1,0,0,3.0,4.0,2.0,3,3,7.373651
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,3,5,1,4,3.0,2.0,4.0,2,4,7.718969
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,2,5,0,3,2.0,2.0,1.0,3,3,7.013240
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,5,8,7,5,4.0,4.0,3.0,2,3,7.193678
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,2,6,0,4,4.0,1.0,3.0,3,3,8.006175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,4406,Female,...,5,3,0,2,4.0,1.0,3.0,3,3,8.522277
4406,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,4407,Male,...,2,3,0,2,4.0,4.0,3.0,2,3,6.092954
4407,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,4408,Male,...,4,4,1,2,1.0,3.0,3.0,3,4,7.706632
4408,42,No,Travel_Rarely,Sales,18,2,Medical,1,4409,Male,...,2,9,7,8,4.0,1.0,3.0,2,3,9.492595
